In [1]:
%load_ext dotenv
%dotenv

In [2]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

class State(TypedDict):
    messages: Annotated[list, add_messages]
    query: str
    response: str
    evaluation: str
    
    
graph_builder = StateGraph(State)

In [3]:
import os
from langchain.chat_models import init_chat_model

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("API_VERSION")

llm = init_chat_model(
    "azure_openai:gpt-4o",
    azure_deployment=os.environ["DEPLOYMENT_NAME"],
)

In [4]:
def agent1(state:State): #英訳する関数（後でノードとして定義し利用する）
    query = state['query']
    
    # Prompt template(書き方2)
    prompt1 = ChatPromptTemplate([
      ('system', 'あなたは、Azure のアーキテクトです、ユーザの実行したいことを実現するために必要なヒアリング項目のリストを作成してください。'),
      ('user', """実現したいこと:{query}""")
    ])

    agent1 = (
        prompt1
        | llm
        | StrOutputParser()
    )
    
    response = agent1.invoke(query)
    print(response)
    return {"query":query,"response":response, "messages": state["messages"] + [("assistant", response)]}

In [5]:
def evaluator(state:State): #英訳する関数（後でノードとして定義し利用する）
    response = state['response']
    
    # Prompt template(書き方2)
    prompt = ChatPromptTemplate([
      ('system', 'あなたはAzureのアーキテクトです。Azure 環境構築時のヒアリング項目のリストが十分であれば yes、そうでなければ no と答えてください。'),
      ('user', """{response}""")
    ])

    agent2 = (
        prompt
        | llm
        | StrOutputParser()
    )

    evaluation = agent2.invoke(response).lower()
    print(evaluation)
    return {"evaluation": evaluation}

In [6]:
def loop_or_end(state):
    # quality_okがFalseなら"再度意図判定(classify)"へ、それ以外は"end"
    return "agent_1" if not state["evaluation"]=="yes" else "end"

In [7]:
workflow = StateGraph(State)

workflow.add_node("agent_1", agent1)
workflow.add_node("evaluator", evaluator)


workflow.add_edge(START, "agent_1")
workflow.add_edge("agent_1", "evaluator")
workflow.add_conditional_edges("evaluator", loop_or_end, {
    "agent_1": "agent_1",
    "end": END
})

graph_app = workflow.compile()

In [8]:
initial_state = State(query='Azure上にWebアプリを作りたい') #Agentに翻訳してほしい言葉を入れる

In [9]:
# streamを利用する場合
for chunk in graph_app.stream(initial_state):
    print(chunk)

# {'agent1': {'query': 'こんにちは', 'answer': 'Hello.'}}


Azure 上に Web アプリを構築、展開するために、具体的な要件を把握し、アーキテクチャや構成を設計するには、以下のような質問をヒアリングリストとして用意することが重要です。

---

### 1. **Webアプリの目的・概要**
- このWebアプリの用途は何ですか？（例: ECサイト、ブログ、業務システム、サポートサイトなど）
- ターゲットユーザーは誰ですか？（一般ユーザー、社内利用者、特定顧客など）
- アプリの主な機能は何ですか？

---

### 2. **トラフィックとスケーリング要件**
- 予想されるユーザー数、または同時接続ユーザー数はどれくらいですか？
- 月間のトラフィック量はどれほどを見込んでいますか？
- トラフィック量に大きな変動はありますか？（例: イベント時、大型キャンペーン）
- 自動スケールのニーズがありますか？（負荷に応じてリソースを増減させたい）

---

### 3. **アプリケーション設計・技術スタック**
- どのプログラミング言語やフレームワークで構築する予定ですか？（例: .NET, Java, Python, Node.js）
- データベースは必要ですか？どんな種類を使用しますか？（リレーショナル [例: Azure SQL Database], NoSQL [例: Cosmos DB]）
- サードパーティのAPIやサービスを利用しますか？

---

### 4. **認証・セキュリティ**
- ユーザー認証は必要ですか？（例: Azure AD, ソーシャルログイン, 独自の認証システム）
- セキュリティ面で特に考慮すべき点はありますか？（例: PCI DSS、GDPR、特定の業界規制）
- HTTPSを使用しますか？

---

### 5. **開発・デプロイメント**
- 開発チームの人数やスキルレベルはどれくらいですか？
- デプロイメントやCI/CDの仕組みをどのように運用しますか？（例: Azure DevOps、GitHub Actions）
- 開発環境と本番環境を分ける必要がありますか？

---

### 6. **パフォーマンス要件**
- 応答時間の目標値はどの程度ですか？（例: 500ms以内）
- エラー率や可用性に関する要求はありますか？（例: 可用性99.9

In [10]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass